# COMP30830 - Software Engineering
# Project: DublinBikes
Group work by:
- **XinHui Jiang** (21200030)
- **Bo Tian** (20211348)
- **Cheng Zhang**(20211342)

# Machine Learning prediction model
In this notebook we develop a ML model to predict two key values that are functional to our web application:
- **number of available bikes** for a selected station
- **number of available bike stands** for a selected station

The model is trained on data fetched from an Amazon RDS database, where we store historical data referring to:
- real-time information that Dublin Bikes makes available through their web API
- Dublin weather information from OpenWeather, obtained by a web API

**N.B.**: all database data are kept up-to-date.

## Getting started
First of all, we need to import Python packages that are required to our model, as well as fetching the data from the Amazon RDS database.
### Import required packages
A number of Python packages are required for our model in order to work, so we import them in our Notebook.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.options.mode.chained_assignment=None
%matplotlib inline
import os
import sqlalchemy as sqla 
from sqlalchemy import create_engine 
import pandas as pd
import sys
import datetime

### Connect to Amazon RDS database
After having stored all the necessary credentials in specific variables, we connect to the database providing error-handling in case of connection issues.

In [3]:
# Declare and initialize database connection credentials
HOST = "dbbikes.cnehrlobjbfz.us-east-1.rds.amazonaws.com"
USERNAME = "tianjiangzhang"
PASSWORD = "bohuicheng"
DATABASE = "dbbikes"
PORT = "3306"

# Attempt connection to database
# Print a statement on the screen to check whether the connection is working
try:
    engine = create_engine("mysql://{}:{}@{}:{}/{}".format(USERNAME, PASSWORD, HOST, PORT, DATABASE), echo=True)
    connection = engine.connect()
    print('+=========================+')
    print('|  CONNECTED TO DATABASE  |')
    print('+=========================+')
    
# Exit if connection not working   
except Exception as e:
        sys.exit(e)

2022-04-06 19:39:14,281 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2022-04-06 19:39:14,283 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-06 19:39:14,429 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2022-04-06 19:39:14,430 INFO sqlalchemy.engine.Engine [generated in 0.00132s] ()
2022-04-06 19:39:14,700 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-04-06 19:39:14,701 INFO sqlalchemy.engine.Engine [raw sql] ()
+=========================+
|  CONNECTED TO DATABASE  |
+=========================+


### Data fetching

#### Bikes information
We store real-time data from Dublin Bikes in a relation named **"occupancyInfo"** in the RDS database schema.

We fetch Pandas dataframe object and we examine it.

In [4]:
# Create dataframe and store data running SQL query
df = pd.read_sql_query("SELECT * FROM dbbikes.occupancyInfo", engine)

# Examine dataframe object, show first 10 rows
df.head(10)

2022-04-06 19:39:22,541 INFO sqlalchemy.engine.Engine SELECT * FROM dbbikes.occupancyInfo
2022-04-06 19:39:22,542 INFO sqlalchemy.engine.Engine [raw sql] ()


,last_update,latitude,longitude,stationNo,stationName,status,banking,bonus,bike_stands,avaiable_bike_stands,avaiable_bikes
0,1.645100e+09,53.3496,-6.27820,42,SMITHFIELD NORTH,OPEN,0,0,30,11,19
1,1.645100e+09,53.3535,-6.26531,30,PARNELL SQUARE NORTH,OPEN,0,0,20,11,9
2,1.645100e+09,53.3360,-6.26298,54,CLONMEL STREET,OPEN,0,0,33,13,20
3,1.645100e+09,53.3594,-6.27614,108,AVONDALE ROAD,OPEN,0,0,35,19,16
4,1.645100e+09,53.3380,-6.24153,56,MOUNT STREET LOWER,OPEN,0,0,40,19,21
5,1.645100e+09,53.3434,-6.27012,6,CHRISTCHURCH PLACE,OPEN,0,0,20,16,4
6,1.645100e+09,53.3341,-6.26544,18,GRANTHAM STREET,OPEN,0,0,30,11,19
7,1.645100e+09,53.3443,-6.25043,32,PEARSE STREET,OPEN,0,0,30,18,12
8,1.645100e+09,53.3388,-6.26200,52,YORK STREET EAST,OPEN,0,0,32,22,10
9,1.645100e+09,53.3478,-6.24424,48,EXCISE WALK,OPEN,0,0,40,17,23


In [5]:
# Show number of rows and columns
print("The dataset has %s rows and %s columns." % df.shape)

The dataset has 525755 rows and 11 columns.


Since 110 stations provide real-time information, we show the first 3 entries in our dataframe (first 110 * 3 = 330 rows)

In [6]:
# Show first 3 real-time bikes entries
df.head(330)

,last_update,latitude,longitude,stationNo,stationName,status,banking,bonus,bike_stands,avaiable_bike_stands,avaiable_bikes
0,1.645100e+09,53.3496,-6.27820,42,SMITHFIELD NORTH,OPEN,0,0,30,11,19
1,1.645100e+09,53.3535,-6.26531,30,PARNELL SQUARE NORTH,OPEN,0,0,20,11,9
2,1.645100e+09,53.3360,-6.26298,54,CLONMEL STREET,OPEN,0,0,33,13,20
3,1.645100e+09,53.3594,-6.27614,108,AVONDALE ROAD,OPEN,0,0,35,19,16
4,1.645100e+09,53.3380,-6.24153,56,MOUNT STREET LOWER,OPEN,0,0,40,19,21
...,...,...,...,...,...,...,...,...,...,...,...
325,1.645270e+09,53.3324,-6.25272,39,WILTON TERRACE,OPEN,0,0,20,13,7
326,1.645270e+09,53.3407,-6.30819,83,EMMET ROAD,OPEN,0,0,40,21,19
327,1.645260e+09,53.3478,-6.29243,92,HEUSTON BRIDGE (NORTH),OPEN,0,0,40,13,27
328,1.645260e+09,53.3422,-6.25449,21,LEINSTER STREET SOUTH,OPEN,0,0,30,12,18


In [8]:
def getMin(df, attr):
    t = datetime.datetime.fromtimestamp(df[attr])
    return t.minute

def getHour(df, attr):
    t = datetime.datetime.fromtimestamp(df[attr])
    return t.hour   
    
def getDate(df, attr):
    t = datetime.datetime.fromtimestamp(df[attr])
    my_date = t.strftime("%Y-%m-%d")
    return my_date

def getDay(df, attr):
    t = datetime.datetime.fromtimestamp(df[attr])
    year = t.year
    month = t.month
    day = t.day
    weekofday = datetime.date(year, month, day).isoweekday()
    return weekofday
    
df['min'] = df.apply(lambda x:getMin(x, 'last_update'),axis=1)
df['hour'] = df.apply(lambda x:getHour(x, 'last_update'),axis=1)
df['date'] = df.apply(lambda x:getDate(x, 'last_update'),axis=1)
df['day'] = df.apply(lambda x:getDay(x, 'last_update'),axis=1)

In [9]:
df[['last_update', 'min', 'hour', 'date', 'day']]

,last_update,min,hour,date,day
0,1.645100e+09,13,12,2022-02-17,4
1,1.645100e+09,13,12,2022-02-17,4
2,1.645100e+09,13,12,2022-02-17,4
3,1.645100e+09,13,12,2022-02-17,4
4,1.645100e+09,13,12,2022-02-17,4
...,...,...,...,...,...
525750,1.649270e+09,33,19,2022-04-06,3
525751,1.649270e+09,33,19,2022-04-06,3
525752,1.649270e+09,33,19,2022-04-06,3
525753,1.649270e+09,33,19,2022-04-06,3


#### Weather information
We store weather data from OpenWeather in a relation named **"weatherInfo"** in the RDS database schema.

We fetch Pandas dataframe object and we examine it.

In [10]:
# Create dataframe and store data running SQL query
df_2 = pd.read_sql_query("SELECT * FROM dbbikes.weatherInfo", engine)

# Examine dataframe object, show first 50 rows
df_2.head(50)

2022-04-06 19:41:50,522 INFO sqlalchemy.engine.Engine SELECT * FROM dbbikes.weatherInfo
2022-04-06 19:41:50,524 INFO sqlalchemy.engine.Engine [raw sql] ()


,currentTime,temperature,feelslikeTemp,humidity,uvi,cloud,visibility,windSpeed,windDegree,weather
0,1646249468,9.18,6.51,92,0.00,75,10000,5.14,130,Rain
1,1646249621,9.18,6.51,92,0.00,75,10000,5.14,130,Rain
2,1646251437,8.66,5.66,92,0.00,75,10000,5.66,110,Rain
3,1646253253,8.91,5.98,88,0.00,75,10000,5.66,210,Drizzle
4,1646255069,8.69,5.90,87,0.00,75,10000,5.14,170,Clouds
5,1646256885,8.74,5.76,85,0.00,75,10000,5.66,180,Clouds
6,1646258702,8.46,5.23,88,0.00,75,10000,6.17,160,Clouds
7,1646260518,8.58,5.56,89,0.00,75,10000,5.66,150,Clouds
8,1646262334,8.49,5.45,89,0.00,75,10000,5.66,150,Clouds
9,1646264150,8.88,6.13,89,0.00,75,10000,5.14,150,Drizzle


In [11]:
# Show number of rows and columns
print("The dataset has %s rows and %s columns." % df_2.shape)

The dataset has 1665 rows and 10 columns.


In [12]:
df_2['min'] = df_2.apply(lambda x:getMin(x, 'currentTime'),axis=1)
df_2['hour'] = df_2.apply(lambda x:getHour(x, 'currentTime'),axis=1)
df_2['date'] = df_2.apply(lambda x:getDate(x, 'currentTime'),axis=1)
# df_2['day'] = df_2.apply(lambda x:getDay(x, 'currentTime'),axis=1)

In [13]:
df_2[['min','hour','date']]

,min,hour,date
0,31,19,2022-03-02
1,33,19,2022-03-02
2,3,20,2022-03-02
3,34,20,2022-03-02
4,4,21,2022-03-02
...,...,...,...
1660,35,17,2022-04-06
1661,5,18,2022-04-06
1662,36,18,2022-04-06
1663,6,19,2022-04-06


In [14]:
df_2.head(220)

,currentTime,temperature,feelslikeTemp,humidity,uvi,cloud,visibility,windSpeed,windDegree,weather,min,hour,date
0,1646249468,9.18,6.51,92,0.00,75,10000,5.14,130,Rain,31,19,2022-03-02
1,1646249621,9.18,6.51,92,0.00,75,10000,5.14,130,Rain,33,19,2022-03-02
2,1646251437,8.66,5.66,92,0.00,75,10000,5.66,110,Rain,3,20,2022-03-02
3,1646253253,8.91,5.98,88,0.00,75,10000,5.66,210,Drizzle,34,20,2022-03-02
4,1646255069,8.69,5.90,87,0.00,75,10000,5.14,170,Clouds,4,21,2022-03-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,1646638390,3.33,-0.82,76,0.21,75,10000,5.14,150,Clouds,33,7,2022-03-07
216,1646640206,4.19,-0.20,75,0.21,75,10000,6.17,150,Clouds,3,8,2022-03-07
217,1646642022,4.48,0.38,73,0.62,75,10000,5.66,160,Clouds,33,8,2022-03-07
218,1646643838,4.81,0.59,71,0.62,75,10000,6.17,130,Clouds,3,9,2022-03-07


## Data preparation
### Unique values functional to merging
We need to **merge the bikes and weather dataframes**.<br/>
Since we have stored in the database a **scraping-timestamp** for each single row in all our relations, we determine the **unique minutes** of fetching (we assume the hours are fully represented 0-24 as data are fetched multiple times within a hour).

In [15]:
print("Unique values for:\n- mins in weather:", pd.unique(df_2['min'].unique()))

Unique values for:
- mins in weather: [31 33  3 34  4  5 35  6 36  7 37  8 38 39  9 40 10 41 11 42 12 13 43 14
 44 15 45 16 46 47 17 48 18 49 19 50 20 51 21 22 52 23 53 24 54 25 55 56
 26 57 27 58 28 59 29  0 30  1 32  2]


In [16]:
print("Unique values for:\n- mins in availability:", pd.unique(df['min'].unique()))

Unique values for:
- mins in availability: [13 40 26  0 46 33 20  6 53]


### Merge dataframes
Now that we have unique values in common between the two dataframes in terms of scraping timestamps (we want the data to be consistent), we **proceed to merge them using a SQL inner join** and store the result in a new dataframe object.

Note the reduction in terms of tuples (rows) from the original bikes dataframe.

In [17]:
# Merge dataframes
# mergedStuff = pd.merge(df, df_2, left_on = ['last_update'], right_on = ['currentTime'], how = 'inner')
mergedStuff = pd.merge(df, df_2, on = ['date', 'hour'], how = 'inner')
# Show first 2 scraps (oldest) for stations (110*2=220)
mergedStuff.head(220)
print(mergedStuff.shape)

(366285, 26)


In [18]:
# Show last 2 scraps (most recent) for stations (110*2=220)
mergedStuff['day'].tail(220)

366065    3
366066    3
366067    3
366068    3
366069    3
         ..
366280    3
366281    3
366282    3
366283    3
366284    3
Name: day, Length: 220, dtype: int64

In [19]:
# Show number of rows and columns of the new joined dataframe
print("The dataset has %s rows and %s columns." % mergedStuff.shape)

The dataset has 366285 rows and 26 columns.


### Dummy coding
We need to incorporate the following information (variables) in our model, but they are not immediately usable as they come in a categorical form:
- **weather (which represents the weather)**
- **day (which represents the day of a week)**

Thus, we transform them in a **series of dummy variables**. The process is known as "dummy coding".
#### Weather: cloud coverage
To incorporate cloud coverage information in our ML model, we need to identify the unique values representing **weather** information and create dummy variables for each.

In [20]:
print("Unique values for:\n- cloud coverage in weather:", pd.unique(mergedStuff['weather'].unique()))

Unique values for:
- cloud coverage in weather: ['Rain' 'Clouds' 'Mist' 'Drizzle' 'Clear' 'Fog']


In [21]:
# print("Unique values for:\n- cloud coverage in weather:", pd.unique(mergedStuff['day'].unique()))

In [22]:
print(mergedStuff['day'].unique())

[3 4 5 6 7 1 2]


#### Timestamp: days of the week
We assume that the unique values for week days are known, we can proceed to dummy coding.

In [23]:
# Create a separate dataframe with days of the week (categorical)
data_input1 = pd.DataFrame(mergedStuff['day'])

# Create a separate dataframe with cloud coverage information (categorical)
data_input2 = pd.DataFrame(mergedStuff['weather'])

# Concatenate the two dataframes in the main one
dummy = pd.get_dummies(data_input1)
dummy_2 = pd.get_dummies(data_input2)
mergedStuff = pd.concat([mergedStuff,dummy],axis=1)
mergedStuff = pd.concat([mergedStuff,dummy_2],axis=1)

In [24]:
# Show last two station scraps and check dummy coded columns
mergedStuff.tail(220)

,last_update,latitude,longitude,stationNo,stationName,status,banking,bonus,bike_stands,avaiable_bike_stands,...,windDegree,weather,min_y,day,weather_Clear,weather_Clouds,weather_Drizzle,weather_Fog,weather_Mist,weather_Rain
366065,1.649270e+09,53.3496,-6.27820,42,SMITHFIELD NORTH,OPEN,0,0,30,0,...,240,Clouds,6,3,0,1,0,0,0,0
366066,1.649270e+09,53.3496,-6.27820,42,SMITHFIELD NORTH,OPEN,0,0,30,0,...,260,Clouds,36,3,0,1,0,0,0,0
366067,1.649270e+09,53.3535,-6.26531,30,PARNELL SQUARE NORTH,OPEN,0,0,20,8,...,240,Clouds,6,3,0,1,0,0,0,0
366068,1.649270e+09,53.3535,-6.26531,30,PARNELL SQUARE NORTH,OPEN,0,0,20,8,...,260,Clouds,36,3,0,1,0,0,0,0
366069,1.649270e+09,53.3360,-6.26298,54,CLONMEL STREET,OPEN,0,0,33,31,...,240,Clouds,6,3,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366280,1.649270e+09,53.3478,-6.29243,92,HEUSTON BRIDGE (NORTH),OPEN,0,0,40,17,...,260,Clouds,36,3,0,1,0,0,0,0
366281,1.649270e+09,53.3422,-6.25449,21,LEINSTER STREET SOUTH,OPEN,0,0,30,19,...,240,Clouds,6,3,0,1,0,0,0,0
366282,1.649270e+09,53.3422,-6.25449,21,LEINSTER STREET SOUTH,OPEN,0,0,30,19,...,260,Clouds,36,3,0,1,0,0,0,0
366283,1.649270e+09,53.3488,-6.28164,88,BLACKHALL PLACE,OPEN,0,0,30,24,...,240,Clouds,6,3,0,1,0,0,0,0


### Check dataframe state
#### New columns
The new dummy coded variables have been concatenated at the end of the dataframe, so we now check our dataframe shape.

In [25]:
# Show number of rows and columns of the dataframe
print("The dataset has %s rows and %s columns." % mergedStuff.shape)

The dataset has 366285 rows and 33 columns.


#### Data types
As a further check, we analyze the data type in our dataframe.

In [26]:
mergedStuff.dtypes

last_update             float64
latitude                float64
longitude               float64
stationNo                 int64
stationName              object
status                   object
banking                   int64
bonus                     int64
bike_stands               int64
avaiable_bike_stands      int64
avaiable_bikes            int64
min_x                     int64
hour                      int64
date                     object
day                       int64
currentTime               int64
temperature             float64
feelslikeTemp           float64
humidity                  int64
uvi                     float64
cloud                     int64
visibility                int64
windSpeed               float64
windDegree                int64
weather                  object
min_y                     int64
day                       int64
weather_Clear             uint8
weather_Clouds            uint8
weather_Drizzle           uint8
weather_Fog               uint8
weather_

## ML model
In order to predict our two target variables
- **number of available bikes**
- **number of available bike stands**
we need two differnt ML models that are trained in the following section.
After having tested the regression model as a viable alternative, we decided to implement a Random Forest classifier model, as it proves to be a more effective predictor.

**N.B.**: we train our models on a random selection of 2/3 of the original dataset. We perform testing on the remaining 1/3. 

### Predict the number of available bikes

#### Model training

In [27]:
# Select model features and store them in a new dataframe
input_model = pd.DataFrame(mergedStuff[['stationNo', 'hour', 'temperature', 'windSpeed', 'feelslikeTemp', 'humidity', 'visibility', 'weather_Clear', 'weather_Clouds', 'weather_Drizzle', 'weather_Fog', 'weather_Mist', 'weather_Rain', 'day']])

# Define target variable
output = mergedStuff['avaiable_bikes']

# Split dataset to train and test
X_train,X_test,Y_train,Y_test=train_test_split(input_model,output,test_size=0.33,random_state=42)
print("Training the model on %s rows and %s columns." % X_train.shape)

# Instantiate RandomForestRegressor object calling 100 decision tree models
rf = RandomForestRegressor(n_estimators=100)

# Train the model
rf.fit(X_train, Y_train)

print("Testing the model on %s rows." % Y_test.shape[0])

Training the model on 245410 rows and 15 columns.
Testing the model on 120875 rows.


#### Model testing

In [28]:
# Get prediction for test cases
prediction = rf.predict(X_test)

# Show the predicted test cases
print(prediction)
# print(Y_test)

[21.27152381  5.60309524  7.85346429 ... 37.39881205  9.56237662
 28.48444048]


#### Model evaluation
In order to evaluate the prediction effectiveness of our model, we first use **accuracy** to see how the model works.Besides, we also compute the **root-mean-square deviation**, defined as the standard deviation of the residuals (prediction errors).

In [29]:
accuracy = rf.score(X_test,Y_test)
print(accuracy)

0.9136731202398845


In [30]:
print("RMSE: %f" % np.sqrt(metrics.mean_squared_error(Y_test,prediction)))

RMSE: 2.689304


#### Module integration
In order to connect the ML model to our Flask web application, we need to produce a 'prediction-data' file from the trained model using the **Pickle** Python module.<br/>
Pickle allows us to store the prediction model in a file that we save on the server, in order to be used by the application to actually deliver a prediction based on the requested stations by the user.

In [31]:
import pickle

pickle.dump(rf, open('final_prediction.pickle', 'wb'))

In [32]:
# This is not strictly functional to the application
random_forest = pickle.load(open("final_prediction.pickle", "rb"))

### Predict the number of available bike stands

#### Model training

In [33]:
# Select model features and store them in a new dataframe
input_model_available_bike_stands = pd.DataFrame(mergedStuff[['stationNo', 'hour', 'temperature', 'windSpeed', 'feelslikeTemp', 'humidity', 'visibility', 'weather_Clear', 'weather_Clouds', 'weather_Drizzle', 'weather_Fog', 'weather_Mist', 'weather_Rain', 'day']])

# Define target variable
output_bike_stands=mergedStuff['avaiable_bike_stands']

# Split dataset to train and test
X_train_2, X_test_2, Y_train_2, Y_test_2 = train_test_split(input_model_available_bike_stands, output_bike_stands, test_size = 0.33, random_state = 42)

print("Training the model on %s rows and %s columns." % X_train_2.shape)

# Instantiate RandomForestRegressor object calling 100 decision tree models
rf_2 = RandomForestRegressor(n_estimators = 100)

# Train the model
rf_2.fit(X_train_2, Y_train_2)

print("Testing the model on %s rows." % Y_test_2.shape[0])

Training the model on 245410 rows and 15 columns.
Testing the model on 120875 rows.


#### Model testing

In [34]:
# Get prediction for test cases
prediction_bike_stands = rf_2.predict(X_test_2)

# Show the predicted test cases
print (prediction_bike_stands)
# print(Y_test_2)

[ 6.44939286 33.69663095 22.3054881  ...  2.65367063  9.0275
 11.5406746 ]
0.9361165544381647


#### Model evaluation
In order to evaluate the prediction effectiveness of our model, we first use **accuracy** to see how the model works.Besides, we also compute the **root-mean-square deviation**, defined as the standard deviation of the residuals (prediction errors)

In [35]:
accuracy2 = rf_2.score(X_test_2,Y_test_2)
print(accuracy2)
print("RMSE: %f" % np.sqrt(metrics.mean_squared_error(Y_test_2,prediction_bike_stands)))

RMSE: 2.731076


#### Module integration
In order to connect the ML model to our Flask web application, we need to produce a 'prediction-data' file from the trained model using the **Pickle** Python module.<br/>
Pickle allows us to store the prediction model in a file that we save on the server, in order to be used by the application to actually deliver a prediction based on the requested stations by the user.

In [36]:
import pickle

pickle.dump(rf_2,open('final_prediction_bike_stands.pickle', 'wb'))

In [37]:
# This is not strictly functional to the application
random_forest_stands=pickle.load(open("final_prediction_bike_stands.pickle", "rb"))